## DISCLAIMER

This is implementation from pyTorch tutorials - https://pytorch.org/tutorials/intermediate/reinforcement_ppo.html

They use some weird convention of naming value network as critic network and policy network as actor network. This is not only the matter of this notebook or their tutorial but also the parameters withing torchrl library

## Importing Packages

In [1]:
import warnings
warnings.filterwarnings("ignore")
from torch import multiprocessing


from collections import defaultdict

import matplotlib.pyplot as plt
import torch
from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor
from torch import nn
from torchrl.collectors import SyncDataCollector
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.envs import (Compose, DoubleToFloat, ObservationNorm, StepCounter,
                          TransformedEnv)
from torchrl.envs.libs.gym import GymEnv
from torchrl.envs.utils import check_env_specs, ExplorationType, set_exploration_type
from torchrl.modules import ProbabilisticActor, TanhNormal, OneHotCategorical, ValueOperator
from torchrl.objectives import ClipPPOLoss
from torchrl.objectives.value import GAE
from tqdm import tqdm

## Hyperparameters

In [2]:
is_fork = multiprocessing.get_start_method() == "fork"
device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)

num_cells = 256  # number of cells in each layer i.e. output dim.
lr = 3e-4
max_grad_norm = 1.0

### Data Collection

In [3]:
# buffer_size = 10000
# batch_size = 1000 # Number of frames per batch
# For a complete training, bring the number of frames up to 1M
total_frames = 500_000 # total size of frames to train on

### PPO parameters

In [4]:
batch_size = 5000 # Number of frames per batch
sub_batch_size = 1000  # cardinality of the sub-samples gathered from the current data in the inner loop
num_epochs = 2  # optimization steps per batch of data collected
clip_epsilon = (
    0.2  # clip value for PPO loss: see the equation in the intro for more context.
)
gamma = 0.99
lmbda = 0.95
entropy_eps = 0.005

## Environment

In [5]:
import gym
import numpy as np

from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.envs.unity_gym_env import UnityToGymWrapper
from torchrl.envs import GymWrapper, TransformedEnv, Compose, ObservationNorm, DoubleToFloat, StepCounter

class UnityGymTorchWrapper(gym.Env):
    """
    Custom wrapper for UnityToGymWrapper to make it compatible with TorchRL's GymWrapper.
    """

    def __init__(self, unity_gym_env):
        super().__init__()
        self.unity_gym_env = unity_gym_env
        self.observation_space = unity_gym_env.observation_space
        self.action_space = unity_gym_env.action_space
        
        # print("init")

    def reset(self):
        observation = self.unity_gym_env.reset()
        # print("observation", len(observation))
        return {"observation": observation}, {}  # TorchRL expects a dictionary

    def step(self, action):
        # print("step")
      
        observation, reward, done, info = self.unity_gym_env.step(action)

        # Split `done` into `terminated` and `truncated`
        terminated = done  # Assuming `done` corresponds to a terminal state
        truncated = False  # Unity environments don't have truncation by default

        return {"observation": observation}, reward, terminated, truncated, info

    def render(self, mode="human"):
        # print("render")
        return self.unity_gym_env.render(mode=mode)

    def close(self):
        # print("close")
        self.unity_gym_env.close()


In [ ]:
channel = EngineConfigurationChannel()
env_path = "C:/Users/Pawel/Documents/Unity_Project/warehouse-bot-training/environment_builds/test_env/Warehouse_Bot.exe"

unity_env = UnityEnvironment(
  file_name=env_path,
  side_channels=[channel],
  additional_args=["-batchmode", "-nographics"]
)
channel.set_configuration_parameters(time_scale=4)

# Convert Unity environment to Gym and wrap it for TorchRL
unity_gym_env = UnityToGymWrapper(unity_env, allow_multiple_obs=True)
custom_env = UnityGymTorchWrapper(unity_gym_env)

# Wrap the custom Gym environment into TorchRL-compatible environment
base_env = GymWrapper(custom_env)

### Environment Normalization

In [7]:
env = TransformedEnv(
    base_env,
    Compose(
        # normalize observations
        ObservationNorm(in_keys=["observation"]),
        DoubleToFloat(),
        StepCounter(),
    ),
)

In [8]:
env.transform[0].init_stats(num_iter=batch_size, reduce_dim=0, cat_dim=0)

## Environment Testing

In [ ]:
print("normalization constant shape:", env.transform[0].loc.shape)

In [ ]:
print("observation_spec:", env.observation_spec)
print("reward_spec:", env.reward_spec)
print("input_spec:", env.input_spec)
print("action_spec (as defined by input_spec):", env.action_spec)

In [ ]:
check_env_specs(env)

In [ ]:
rollout = env.rollout(3)
print("rollout of three steps:", rollout)
print("Shape of the rollout TensorDict:", rollout.batch_size)

## Policy Network

In [13]:
policy_net = nn.Sequential(
    nn.LazyLinear(num_cells, device=device),
    nn.Tanh(),
    nn.LazyLinear(num_cells, device=device),
    nn.Tanh(),
    nn.LazyLinear(num_cells, device=device),
    nn.Tanh(),
    nn.LazyLinear(2 * env.action_spec.shape[-1], device=device),
    NormalParamExtractor(),
).to(device)

In [14]:
policy_module = TensorDictModule(
    policy_net, in_keys=["observation"], out_keys=["logits"] #out_keys=["loc", "scale"]
)

In [15]:
policy_module = ProbabilisticActor(
    module=policy_module,
    spec=env.action_spec,
    in_keys=["logits"],
    distribution_class=OneHotCategorical,
    distribution_kwargs={},
    return_log_prob=True,
    # we'll need the log-prob for the numerator of the importance weights
).to(device)

## Value Network

In [16]:
value_net = nn.Sequential(
    nn.LazyLinear(num_cells, device=device),
    nn.Tanh(),
    nn.LazyLinear(num_cells, device=device),
    nn.Tanh(),
    nn.LazyLinear(num_cells, device=device),
    nn.Tanh(),
    nn.LazyLinear(1, device=device),
    nn.Flatten() # ??? VERIFY
)

In [17]:
value_module = ValueOperator(
    module=value_net,
    in_keys=["observation"],
).to(device)

## Tesitng Policy and Value Modules

In [ ]:
print("Running policy:", policy_module(env.reset().to(device)))
print("Running value:", value_module(env.reset().to(device)))


## Data Collector

In [19]:
collector = SyncDataCollector(
    env,
    policy_module,
    frames_per_batch=batch_size,
    total_frames=total_frames,
    split_trajs=False,
    device=device,
    reset_when_done=True, # ??? VERIFY
    reset_at_each_iter=True, # ??? VERIFY
)

## Replay Buffer

In [20]:
replay_buffer = ReplayBuffer(
    storage=LazyTensorStorage(max_size=batch_size),
    sampler=SamplerWithoutReplacement(),
)

## Loss Function

In [21]:
# GAE - Generalized Advantage Estimation
advantage_module = GAE(
    gamma=gamma, lmbda=lmbda, value_network=value_module, average_gae=True
)

loss_module = ClipPPOLoss(
    actor_network=policy_module,
    critic_network=value_module,
    clip_epsilon=clip_epsilon,
    entropy_bonus=bool(entropy_eps),
    entropy_coef=entropy_eps,
    # these keys match by default but we set this for completeness
    critic_coef=1.0,
    loss_critic_type="smooth_l1",
)

optim = torch.optim.Adam(loss_module.parameters(), lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optim, total_frames // batch_size, 0.0
)

## Trainign Loop

In [ ]:
print("Policy Device:", next(policy_module.parameters()).device)
print("Environment Device:", env.device)

In [23]:
from torchrl.envs.utils import LazyStackedTensorDict

logs = defaultdict(list)
pbar = tqdm(total=total_frames)
eval_str = ""

# We iterate over the collector until it reaches the total number of frames it was
# designed to collect:
for i, tensordict_data in enumerate(collector):
    # we now have a batch of data to work with. Let's learn something from it.

    for _ in range(num_epochs):
        # We'll need an "advantage" signal to make PPO work.
        # We re-compute it at each epoch as its value depends on the value
        # network which is updated in the inner loop.

        # Compute advantage
        advantage_module(tensordict_data)
        
        data_view = tensordict_data.reshape(-1)
        replay_buffer.extend(data_view.cpu())
        for _ in range(batch_size // sub_batch_size):
            subdata = replay_buffer.sample(sub_batch_size)
            loss_vals = loss_module(subdata.to(device))
            loss_value = (
                loss_vals["loss_objective"]
                + loss_vals["loss_critic"]
                + loss_vals["loss_entropy"]
            )

            # Optimization: backward, grad clipping and optimization step
            loss_value.backward()
            # this is not strictly mandatory but it's good practice to keep
            # your gradient norm bounded
            torch.nn.utils.clip_grad_norm_(loss_module.parameters(), max_grad_norm)
            optim.step()
            optim.zero_grad()

    # Loggs
    logs["reward"].append(tensordict_data["next", "reward"].mean().item())
    cum_reward_str = (
        f"average reward={logs['reward'][-1]: 4.4f} (init={logs['reward'][0]: 4.4f})"
    )
    logs["step_count"].append(tensordict_data["step_count"].max().item())
    stepcount_str = f"step count (max): {logs['step_count'][-1]}"
    logs["lr"].append(optim.param_groups[0]["lr"])
    lr_str = f"lr policy: {logs['lr'][-1]: 4.4f}"
    # ---
    
    if i % 1 == 0:
        # We evaluate the policy once every 10 batches of data.
        # Evaluation is rather simple: execute the policy without exploration
        # (take the expected value of the action distribution) for a given
        # number of steps (1000, which is our ``env`` horizon).
        # The ``rollout`` method of the ``env`` can take a policy as argument:
        # it will then execute this policy at each step.
        
        print("evaluate ", i)
        
        with set_exploration_type(ExplorationType.MEAN), torch.no_grad():
            # execute a rollout with the trained policy
            eval_rollout = env.rollout(batch_size, policy_module)
            
            logs["eval reward"].append(eval_rollout["next", "reward"].mean().item())
            logs["eval reward (sum)"].append(
                eval_rollout["next", "reward"].sum().item()
            )
            logs["eval step_count"].append(eval_rollout["step_count"].max().item())
            eval_str = (
                f"eval cumulative reward: {logs['eval reward (sum)'][-1]: 4.4f} "
                f"(init: {logs['eval reward (sum)'][0]: 4.4f}), "
                f"eval step-count: {logs['eval step_count'][-1]}"
            )
            del eval_rollout
    pbar.set_description(", ".join([eval_str, cum_reward_str, stepcount_str, lr_str]))
    pbar.update(tensordict_data.numel())
    
    # We're also using a learning rate scheduler. Like the gradient clipping,
    # this is a nice-to-have but nothing necessary for PPO to work.
    scheduler.step()

KeyboardInterrupt: 

In [ ]:
env.action_spec.space

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(logs["reward"])
plt.title("training rewards (average)")
plt.subplot(2, 2, 2)
plt.plot(logs["step_count"])
plt.title("Max step count (training)")
plt.subplot(2, 2, 3)
plt.plot(logs["eval reward (sum)"])
plt.title("Return (test)")
plt.subplot(2, 2, 4)
plt.plot(logs["eval step_count"])
plt.title("Max step count (test)")
plt.show()